# Scrapping linkedin
Trying selenium to scrap linkedin.

In [5]:
from selenium import webdriver
import time
import pandas as pd

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [26]:
def initialize_browser():
    
    # browser must be closed for it to work
    
    path_chromedriver = "G:/Downloads/chromedriver.exe"
    path_profile = "user-data-dir=C:\\Users\\Ruben\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 2"
    url = "https://www.linkedin.com/jobs/search?keywords=data%20scientist&location=germany&geoId=&trk=guest_homepage-basic_jobs-search-bar_search-submit&position=1&pageNum=0"
    
    options = Options()
    options.add_argument(path_profile)
    browser = webdriver.Chrome(path_chromedriver, chrome_options=options)
    
    # no idea what we need to do this twice, potentially for aboiding sign in
    for _ in range(2):
        browser.get(url)
    
    return browser

def expand_menu_window(browser, N=10):
    
    print("this can take a while...")
    i = 1
    while i <= N:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        i += 1
        try:
            browser.find_element_by_xpath("/html/body/div/div/main/section[2]/button").click()
            
        except:
            pass
        time.sleep(2)
        
def get_list_of_jobs(browser):
    
    job_lists = browser.find_element_by_class_name("jobs-search__results-list")
    jobs = job_lists.find_elements_by_tag_name("li")
    return jobs

def scrap_jobs(browser, job):
    
    for item, job in enumerate(jobs):
    
        # retrieve info in menu job
        job_title = job.find_element_by_css_selector("h3").get_attribute("innerText")
        company_name = job.find_element_by_css_selector("h4").get_attribute("innerText")
        location = job.find_element_by_css_selector('[class="job-search-card__location"]').get_attribute("innerText")
        submeta = job.find_element_by_css_selector("div.base-search-card__metadata")     
        listing = submeta.find_element_by_tag_name("time")
        listing_date = listing.get_attribute("datetime")
                    
        job_link = job.find_element_by_css_selector("a").get_attribute("href")

        # click on the selected job
        job_click_path = f"div/a"
        try:
            job_click = job.find_element_by_xpath(job_click_path).click()
            time.sleep(2)

            # click on the selected show more
            buttons = browser.find_elements(By.TAG_NAME , "button")
            names = []
            for name in buttons:
                if name.text == "Show more":
                    button = name
            try:
                button.click()
            except:
                print("Could not expand job description")
            time.sleep(2)

            # retrieve job description
            jb_path = "div.show-more-less-html__markup"
            jb = browser.find_element_by_css_selector(jb_path).text   

            # retrieve seniority
            path_list = "ul.description__job-criteria-list"
            more_el = browser.find_element_by_css_selector(path_list)
            elements = more_el.find_elements_by_tag_name("span")
            if len(elements) == 4:
                seniority = elements[0].text
                employement_type = elements[1].text
                job_function = elements[2].text
                industries = elements[3].text
            else:
                seniority = ""
                employement_type = ""
                job_function = ""
                industries = ""
        except:
            pass

        yield job_title, company_name, location, job_link, listing_date, jb, seniority, employement_type, job_function, industries

In [11]:
browser = initialize_browser()

<ipython-input-10-4399fb030f28>:11: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(path_chromedriver, chrome_options=options)


In [16]:
expand_menu_window(browser, N=5)

this can take a while...


In [17]:
jobs = get_list_of_jobs(browser)
print(len(jobs))

74


In [27]:
res = list(scrap_jobs(browser, jobs))

Could not expand job description


In [23]:
data = pd.DataFrame(res)
data.columns = ["job_title", "company", "location", "link", "listing_date", "description", "seniority", "employment_type", "job_function", "industries"]

In [25]:
data.head()

,job_title,company,location,link,listing_date,description,seniority,employment_type,job_function,industries
0,"Data Scientist, International Analytics",DoorDash,"Berlin, Berlin, Germany",https://de.linkedin.com/jobs/view/data-scienti...,23 hours ago,About The Role\n\nThe Analytics team is lookin...,Entry level,Internship,Engineering and Information Technology,Internet
1,Data Scientist (m/f/d),adidas,"Herzogenaurach, Bavaria, Germany",https://de.linkedin.com/jobs/view/data-scienti...,4 weeks ago,General Purpose\nAs Data Scientist you support...,Not Applicable,Full-time,"Advertising, Art/Creative, and Marketing","Consumer Goods, Retail, and Sporting Goods"
2,"Data Scientist, Business Operations",DoorDash,"Berlin, Berlin, Germany",https://de.linkedin.com/jobs/view/data-scienti...,2 days ago,The Business Operations Analytics team drives ...,Entry level,Full-time,Engineering and Information Technology,Internet
3,Data Scientist (m/f/d),adidas,"Herzogenaurach, Bavaria, Germany",https://de.linkedin.com/jobs/view/data-scienti...,2 weeks ago,Purpose & Overall Relevance For The Organizati...,Not Applicable,Full-time,Engineering and Information Technology,"Consumer Goods, Retail, and Sporting Goods"
4,Data Scientist (m/w/d),Temedica,"Munich, Bavaria, Germany",https://de.linkedin.com/jobs/view/data-scienti...,10 hours ago,Read Job Description In\n\nEinleitung\n\nEin T...,Entry level,Full-time,Engineering and Information Technology,"Marketing and Advertising, Information Technol..."


In [24]:
data.to_excel("scrap_DS_2.xlsx")